In [1]:
import os
import gc
import cv2
import time
import random
import glob
from PIL import Image
import  matplotlib.pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from pytorch_toolbelt import losses as L
from pytorch_toolbelt.inference import tta

# Utils
from tqdm.auto import tqdm

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

## using gpu:1
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

In [2]:
class Customize_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=True)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.fc = nn.Linear(in_features, cls)
        
    def forward(self, image):
        x = self.model(image)
        x = self.fc(x)
        return x

In [3]:
def get_test_transform(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        ToTensorV2(p=1.0),
    ])


def inference(model, img_path):
    
    ## preprocessing
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
    img= img.astype(np.float32)
    aug= get_test_transform(CFG['img_size'])
    img= aug(image= img)["image"].cuda()
    img= torch.unsqueeze(img, 0)
    
    for i, m in enumerate(model):
        with torch.no_grad():
            m.eval()
            if CFG['TTA']:
                pred= tta.fliplr_image2label(m, img)[0]
            else:
                pred= m(img)[0]
                
        if i==0: preds= pred
        else: preds+= pred
            
    pred= preds/len(model)
    pred_prob= pred.softmax(dim=-1).cpu().numpy()
    pred= pred.softmax(dim=-1).cpu().argmax(0).numpy()
    return pred, pred_prob

# CFG

In [4]:
CFG= {
    'img_size': int(448*1.1),
    'TTA': True,
    'model_path': './Model/test/effb4_448',
    'model': None,
}
CFG['model']= [ torch.load(m, map_location= 'cuda:0') for m in glob.glob(CFG['model_path']+'/**')]
print(f"length of model: {len(CFG['model'])}")

length of model: 3


# Prepare Dataset

In [5]:
test_df= pd.DataFrame(columns= ('image_path',))
test_df['image_path']= glob.glob('Dataset/test_img_512/**/*jpg', recursive=True)
print(f'test dataset: {len(test_df)}')
test_df.head()

test dataset: 20000


,image_path
0,Dataset/test_img_512\test_0\0005479429a4.jpg
1,Dataset/test_img_512\test_0\0006849c44f2.jpg
2,Dataset/test_img_512\test_0\0006fcc93fc9.jpg
3,Dataset/test_img_512\test_0\00088803914c.jpg
4,Dataset/test_img_512\test_0\000b572940a1.jpg


# Inference

In [ ]:
test_df['pred_num']= None
for i in tqdm(range(len(test_df))):
    img_path= test_df.loc[i, 'image_path']
    pred, pred_prob= inference(CFG['model'], img_path)
    test_df.loc[i, 'pred_num']= pred
test_df.to_csv(f"submission/{CFG['model_path'].split('/')[-1]}.csv", index= False)
test_df.head()